In [ ]:
import typing
from effectful.handlers.llm.template import Template
from effectful.ops.types import NotHandled
from typing import Callable

In [9]:
@Template.define
def generate_paragraph() -> str:
    """Please generate a paragraph: with exactly 4 sentences ending with 'walk', 'tumbling', 'another', and 'lunatic'.
    """
    raise NotHandled

In [20]:
@Template.define
def codeact(
    template_name: str,
    args_json: str = "[]",
    kwargs_json: str = "{}",
) -> Callable[[], str]:
    """Generate a code that solve the following problem:
    {template_name}
    Args/kwargs are provided as JSON strings (args_json, kwargs_json).
    DO NOT USE codeadapt tool.
    """
    raise NotHandled


@Template.define
def codeadapt(
    template_name: str,
    args_json: str = "[]",
    kwargs_json: str = "{}",
) -> str:
    """Reason about the template, uses the codeact tool to generate a code that solve the problem.
    The template:
    {template_name}
    Args/kwargs are provided as JSON strings (args_json, kwargs_json).
    """
    raise NotHandled



In [17]:
import inspect
import json

from effectful.handlers.llm.completions import (
    DecodedToolCall,
    Encodable,
    LiteLLMProvider,
    Message,
    RetryLLMHandler,
    call_assistant,
    call_system,
    call_tool,
    call_user,
    get_message_sequence,
    handler,
    implements,
)
from effectful.handlers.llm.evaluation import RestrictedEvalProvider


class ToolNotUsedError(Exception):
    """Exception raised when a tool is not used in the code."""

    tool_name: str


class CodeAdapt(LiteLLMProvider):
    def __init__(self, model: str = "gpt-4o"):
        super().__init__(model=model)

    @implements(Template.__apply__)
    def _call[**P, T](self, template: Template[P, T], *args: P.args, **kwargs: P.kwargs) -> T:
        tool_called = False

        # Avoid recursive handling when codeadapt is invoked on a Template argument.
        if template is codeadapt and args and isinstance(args[0], Template):
            args = (args[0].__name__, *args[1:])

        message_sequence = get_message_sequence()
        with handler({get_message_sequence: lambda: message_sequence}), handler(
            RetryLLMHandler()
        ):
            # encode arguments
            bound_args = inspect.signature(template).bind(*args, **kwargs)
            bound_args.apply_defaults()
            env = template.__context__.new_child(bound_args.arguments)

            # Create response_model with env so tools passed as arguments are available
            response_model = Encodable.define(
                template.__signature__.return_annotation, env
            )

            call_system(template)
            message: Message = call_user(template.__prompt_template__, env)

            # loop based on: https://cookbook.openai.com/examples/reasoning_function_calls
            tool_calls: list[DecodedToolCall] = []
            result: T | None = None
            while message["role"] != "assistant" or tool_calls:
                print(json.dumps(message_sequence, indent=2))
                message, tool_calls, result = call_assistant(
                    template.tools, response_model, **self.config
                )
                for tool_call in tool_calls:
                    message = call_tool(tool_call)
                    tool_called = True

            assert result is not None, (
                "call_assistant did not produce a result nor tool_calls"
            )
            assert tool_called, "No tool was called"
        return result


In [26]:
from effectful.handlers.llm.evaluation import UnsafeEvalProvider
import litellm

litellm._turn_on_debug()

code_adapt = CodeAdapt(model="gpt-4o")
with handler(LiteLLMProvider(model="gpt-4o")), handler(UnsafeEvalProvider()):
    res = codeadapt("generate_paragraph")
    print(res)

10:21:29 - LiteLLM:DEBUG: utils.py:381 - 

10:21:29 - LiteLLM:DEBUG: utils.py:381 - Request to litellm:
10:21:29 - LiteLLM:DEBUG: utils.py:381 - litellm.completion('gpt-4o', messages=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Reason about the template, uses the codeact tool to generate a code that solve the problem.\n    The template:\n    generate_paragraph\n    Args/kwargs are provided as JSON strings (args_json, kwargs_json).\n'}], 'id': '03acfb74-0114-11f1-a9bf-b15022a67ef9'}], response_format=<class 'effectful.handlers.llm.completions.Response'>, tools=[{'type': 'function', 'function': {'name': 'generate_paragraph', 'description': "Please generate a paragraph: with exactly 4 sentences ending with 'walk', 'tumbling', 'another', and 'lunatic'.\n", 'parameters': {'additionalProperties': False, 'properties': {}, 'title': 'Params', 'type': 'object', 'required': []}, 'strict': True}}, {'type': 'function', 'function': {'name': 'codeact', 'description': 'Generate a code that 

Here is the generated code to solve the problem of creating a paragraph with sentences ending in specific words:

```python
def generate_paragraph() -> str:
    """
    Generates a paragraph with exactly 4 sentences ending with specified words.
    Returns:
        str: A paragraph with sentences ending in 'walk', 'tumbling', 'another', and 'lunatic'.
    """
    return ("The park was a serene sanctuary where she enjoyed her daily walk. "
            "As she played on the grassy slopes, joyful laughter filled the air as she went tumbling. "
            "She often found solace in a world so lively yet always searching for another. "
            "The night, however, revealed a different side of the city, where the streets whispered the tales of a passing lunatic.")
```

This function will generate a paragraph composed of four sentences, each ending with the specified words: "walk", "tumbling", "another", and "lunatic".


10:21:39 - LiteLLM:DEBUG: litellm_logging.py:1402 - response_cost: 0.0033050000000000006
10:21:39 - LiteLLM:DEBUG: utils.py:4830 - checking potential_model_names in litellm.model_cost: {'split_model': 'gpt-4o-2024-08-06', 'combined_model_name': 'openai/gpt-4o-2024-08-06', 'stripped_model_name': 'gpt-4o-2024-08-06', 'combined_stripped_model_name': 'openai/gpt-4o-2024-08-06', 'custom_llm_provider': 'openai'}
10:21:39 - LiteLLM:DEBUG: utils.py:5171 - model_info: {'key': 'gpt-4o-2024-08-06', 'max_tokens': 16384, 'max_input_tokens': 128000, 'max_output_tokens': 16384, 'input_cost_per_token': 2.5e-06, 'input_cost_per_token_flex': None, 'input_cost_per_token_priority': None, 'cache_creation_input_token_cost': None, 'cache_creation_input_token_cost_above_200k_tokens': None, 'cache_read_input_token_cost': 1.25e-06, 'cache_read_input_token_cost_above_200k_tokens': None, 'cache_read_input_token_cost_flex': None, 'cache_read_input_token_cost_priority': None, 'cache_creation_input_token_cost_above_

10:21:39 - LiteLLM:DEBUG: utils.py:5171 - model_info: {'key': 'gpt-4o-2024-08-06', 'max_tokens': 16384, 'max_input_tokens': 128000, 'max_output_tokens': 16384, 'input_cost_per_token': 2.5e-06, 'input_cost_per_token_flex': None, 'input_cost_per_token_priority': None, 'cache_creation_input_token_cost': None, 'cache_creation_input_token_cost_above_200k_tokens': None, 'cache_read_input_token_cost': 1.25e-06, 'cache_read_input_token_cost_above_200k_tokens': None, 'cache_read_input_token_cost_flex': None, 'cache_read_input_token_cost_priority': None, 'cache_creation_input_token_cost_above_1hr': None, 'input_cost_per_character': None, 'input_cost_per_token_above_128k_tokens': None, 'input_cost_per_token_above_200k_tokens': None, 'input_cost_per_query': None, 'input_cost_per_second': None, 'input_cost_per_audio_token': None, 'input_cost_per_token_batches': 1.25e-06, 'output_cost_per_token_batches': 5e-06, 'output_cost_per_token': 1e-05, 'output_cost_per_token_flex': None, 'output_cost_per_toke